In [1]:
import os
from tqdm import tqdm as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

In [2]:
torch.cuda.init()
device = torch.device('cuda:1')
print(torch.cuda.get_device_name(device))

GeForce GTX TITAN X


In [3]:
base_path = '/ssd2/chetanp'
batch_size = 2 ** 15

In [4]:
dataset = []
for i in range(2):
    path = os.path.join(base_path,'brown_e_base'+str(i+4)+'.pt')
    data = torch.load(path)
    dataset.append(TensorDataset(data))
dataset = ConcatDataset(dataset)
data_loader = DataLoader(dataset,batch_size = batch_size,num_workers = 14)

In [5]:
criterion = nn.CosineSimilarity()
loss_function = lambda x,y: (1-criterion(x,y)).mean()
val_losses = []

In [8]:
for i in range (7):
    
    class AutoEncoder(nn.Module):

        def __init__(self):
            super(AutoEncoder, self).__init__() 

            #encode
            self.e1 = nn.Linear(9216,2048)
            self.e2 = nn.Linear(2048,2 ** (5 + i))

            #decode
            self.d1 = nn.Linear(2 ** (5 + i),2048)
            self.d2 = nn.Linear(2048,9216)

        def forward(self, x):
            encode = self.e2(F.relu(self.e1(x)))

            return self.d2(F.relu(self.d1(encode)))
        
    model = nn.DataParallel(AutoEncoder(),device_ids =[1,2]).to(device)
    path = os.path.join(base_path,f'brown_e_base_compressor_{i}.pth')
    model.load_state_dict(torch.load(path))
    model.eval()
    
    running_loss = 0
    count = 0

    for data in tqdm(data_loader):
        ## Get Data
        sample = data[0].to(device)

        ## Pass forward
        output = model(sample)
        loss = loss_function(output,sample)

        ## Update Running Loss
        running_loss += float(loss)
        count += 1

    print(f'Test Set Loss {i}: ', str(running_loss/count))
    val_losses.append(running_loss/count)

100%|██████████| 9/9 [00:12<00:00,  1.02s/it]


Test Set Loss 0:  0.2285224629773034


 33%|███▎      | 3/9 [00:07<00:18,  3.10s/it]


KeyboardInterrupt: 

In [7]:
import csv

path = os.path.join(base_path, 'val_loss.csv')

with open(path, 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(val_losses)